## 1. 1197 최소 스패닝 트리
#### 문제
그래프가 주어졌을 때, 그 그래프의 최소 스패닝 트리를 구하는 프로그램을 작성하시오.  
  
최소 스패닝 트리는, 주어진 그래프의 모든 정점들을 연결하는 부분 그래프 중에서 그 가중치의 합이 최소인 트리를 말한다.  
  
#### 입력
첫째 줄에 정점의 개수 V(1 ≤ V ≤ 10,000)와 간선의 개수 E(1 ≤ E ≤ 100,000)가 주어진다.  
다음 E개의 줄에는 각 간선에 대한 정보를 나타내는 세 정수 A, B, C가 주어진다.  
이는 A번 정점과 B번 정점이 가중치 C인 간선으로 연결되어 있다는 의미이다. C는 음수일 수도 있으며, 절댓값이 1,000,000을 넘지 않는다.  
  
최소 스패닝 트리의 가중치가 -2147483648보다 크거나 같고, 2147483647보다 작거나 같은 데이터만 입력으로 주어진다.  
#### 출력
첫째 줄에 최소 스패닝 트리의 가중치를 출력한다.  
  
#### 예제 입력 1
```
3 3
1 2 1
2 3 2
1 3 3
```
#### 예제 출력 1 
```
3
```

#### ------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### 1-1 크루스칼(Kruskal) 알고리즘 이용 풀이
**사이클을 만들지 않는 가장 가중치가 작은 edge를 추가해가기**
1. 간선을 가중치 기준으로 정렬
2. 작은 간선부터 보면서 사이클을 만들지 않으면 Spanning Tree에 추가.

In [3]:
def find(u):
    if u != p[u]:
        p[u] = find(p[u])
    return p[u]

def union(u, v):
    r1 = find(u)
    r2 = find(v)
    if r1 != r2:
        p[r2] = r1

V, E = map(int, input().split())
edges = sorted([tuple(map(int, input().split())) for _ in range(E)], key=lambda x: x[2])
p = [*range(V+1)]
cost = 0
for u, v, c in edges:
    if find(u) != find(v):
        union(u, v)
        cost += c      
print(cost) 

3 3
1 2 1
2 3 2
1 3 3
3


백준은 입력 받을때 sys.stdin.readline으로 해주기..오랜만에 해서 당황했네

최종코드..

In [ ]:
import sys
input = sys.stdin.readline

def find(u):
    if u != p[u]:
        p[u] = find(p[u])
    return p[u]

V, E = map(int, input().split())
edges = sorted((tuple(map(int, input().split())) for _ in range(E)), key=lambda x: x[2])
p = [*range(V+1)]
cost = e = 0
for u, v, c in edges:
    r1, r2 = find(u), find(v)
    if r1 != r2:
        p[r2] = r1
        cost += c
        e += 1
        if e == V - 1:
            break
print(cost) 

### 1-2. 프림(Prim) 알고리즘 이용 풀이
**현재의 Spanning Tree에서 가장 가까운 정점을 추가해가기**
1. 우선순위 큐에 시작 노드를 넣음
2. 인접한 노드를 우선순위 큐에 넣음
3. 가장 가까운 노드를 넣고 그 노드의 인접한 애들을 다시 우선순위 큐에 넣음. 반복.

In [6]:
# 힙에 추가하기
def heappush(node, cost):
    heap.append((node, cost))
    upheap()

# 가장 가까운 노드 반환
def heappop():
    heap[0], heap[-1] = heap[-1], heap[0]
    k = heap.pop()
    downheap()
    return k

# 0번 인덱스부터 내려가면서 힙 속성 회복. 
# i번 인덱스의 자식은 2 * i + 1, 2 * i + 2
def downheap():
    n = len(heap)
    # 자식이 있는동안
    i = 0  # 루트에서 시작.
    while 2 * i + 1 < n:
        # swap할 인덱스
        k = 2 * i + 1
        # 오른쪽 자식이 있고 cost가 더 작다면 바꿔줌.
        if k + 1 < n and heap[k][1] > heap[k + 1][1]:
            k += 1
        # 자식노드의 cost가 더 작다면 swap
        if heap[i][1] > heap[k][1]:
            heap[i], heap[k] = heap[k], heap[i]
            i = k
        # 아니면 힙속성 만족
        else:
            break
    
    
# 마지막 원소부터 올라가면서 힙 속성 회복
# j번 인덱스의 부모는 (j - 1) // 2
def upheap():
    i = len(heap) - 1  # 마지막 원소 인덱스
    while (i - 1) // 2 >= 0:   # i가 0이 아닌 동안..부모를 찾아가며 비교.
        k = (i - 1) // 2  # 부모 노드의 인덱스
        # 추가한 노드의 가중치가 부모 노드의 가중치보다 작다면 swap
        if heap[i][1] < heap[k][1]:
            heap[i], heap[k] = heap[k], heap[i]
            i = k
        else:
            break        
    

V, E = map(int, input().split())
adj_lst = [[] for _ in range(V + 1)]
for u, v, c in (map(int, input().split()) for _ in range(E)):
    adj_lst[u].append((v, c))
    adj_lst[v].append((u, c))
visit = [0] * (V + 1)
visit[1] = 1
heap = []
for u, cost in adj_lst[1]:
    heappush(u, cost)
selected = 1
ans = 0
while selected < V:    # V개 모두 선택되면 끝.
    # 방문하지 않은 노드중에 최소 가중치 선택
    nxt, cst = heappop()
    while visit[nxt]:
        nxt, cst = heappop()
    ans += cst
    selected += 1
    visit[nxt] = 1
    for u, cost in adj_lst[nxt]:
        if not visit[u]:
            heappush(u, cost)
print(ans)

5 6
1 3 4
1 5 3
1 4 4
4 5 1
2 5 9
2 3 7
15


```
5 6
1 3 4
1 5 3
1 4 4
4 5 1
2 5 9
2 3 7
```

처음에 pop할때 visit여부를 확인하지 않아서 틀렸다.  
이거만 고쳐주니 맞았는데 조금 비효율적인듯..?? 원래 크루스칼보다 느린가?  
heapsize를 계속 계산해서 그런가..ㅜㅜ

Use Prim's algorithm when you have a graph with lots of edges.  
  
For a graph with V vertices E edges, Kruskal's algorithm runs in O(E log V) time and Prim's algorithm can run in O(E + V log V) amortized time, if you use a Fibonacci Heap.  
  
Prim's algorithm is significantly faster in the limit when you've got a really dense graph with many more edges than vertices.  
Kruskal performs better in typical situations (sparse graphs) because it uses simpler data structures.  

## 2. 1916 최소비용 구하기
#### 문제
N의 도시가 있다. 그리고 한 도시에서 출발하여 다른 도시에 도착하는 M개의 버스가 있다.  
우리는 A번째 도시에서 B번째 도시까지 가는데 드는 버스 비용을 최소화 시키려고 한다.  
그러면 A번째 도시에서 B번째 도시까지 가는데 드는 최소비용을 출력하여라. 도시의 번호는 1부터 N까지이다.  
  
#### 입력
첫째 줄에 도시의 개수 N(1 ≤ N ≤ 1,000)이 주어지고 둘째 줄에는 버스의 개수 M(1 ≤ M ≤ 100,000)이 주어진다.  
그리고 셋째 줄부터 M+2줄까지 다음과 같은 버스의 정보가 주어진다. 먼저 처음에는 그 버스의 출발 도시의 번호가 주어진다.  
그리고 그 다음에는 도착지의 도시 번호가 주어지고 또 그 버스 비용이 주어진다. 버스 비용은 0보다 크거나 같고, 100,000보다 작은 정수이다.   
그리고 M+3째 줄에는 우리가 구하고자 하는 구간 출발점의 도시번호와 도착점의 도시번호가 주어진다.  
출발점에서 도착점을 갈 수 있는 경우만 입력으로 주어진다.  
  
#### 출력
첫째 줄에 출발 도시에서 도착 도시까지 가는데 드는 최소 비용을 출력한다.  
  
#### 예제 입력 1 
```
5
8
1 2 2
1 3 3
1 4 1
1 5 10
2 4 2
3 4 1
3 5 1
4 5 3
1 5
```
#### 예제 출력 1 
```
4
```

#### ------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### 2-1. 다익스트라 알고리즘 이용 풀이
D[i]에 출발점으로부터 정점 i까지의 거리(경로의 길이)가 저장.  
다익스트라 알고리즘은 입력 그래프의 가중치가 모두 양수인 경우만 가능.  
```
1. D를 무한대로 초기화. 단 D[s] = 0으로 초기화
2. for k in range(N):
3.   방문 안된 각 정점 i에 대해 D[i]가 최소인 정점 min_vertex를 찾고, 방문
4.   for min_vertex에 인접한 각 정점 w에 대해서:
5.     if w가 방문 안된 정점이면:
6.       if D[min_vertext] + 간선(min_vertext, w)의 가중치 < D[w]:
7.         D[w] = D[min_vertex] + 간선 (min_vertex, w)의 가중치   # 간선완화
8.         previous[w] = min_vertex
```

시작정점만 0으로 초기화 해주고 인접한 정점 i에 대해 D[i]가 갱신되는 경우만 우선순위 큐에 추가해준다.

In [20]:
# 힙에 추가하기
def heappush(node, cost):
    heap.append((node, cost))
    upheap()

# 가장 가까운 노드 반환
def heappop():
    heap[0], heap[-1] = heap[-1], heap[0]
    k = heap.pop()
    downheap()
    return k

# 0번 인덱스부터 내려가면서 힙 속성 회복. 
# i번 인덱스의 자식은 2 * i + 1, 2 * i + 2
def downheap():
    n = len(heap)
    # 자식이 있는동안
    i = 0  # 루트에서 시작.
    while 2 * i + 1 < n:
        # swap할 인덱스
        k = 2 * i + 1
        # 오른쪽 자식이 있고 cost가 더 작다면 바꿔줌.
        if k + 1 < n and heap[k][1] > heap[k + 1][1]:
            k += 1
        # 자식노드의 cost가 더 작다면 swap
        if heap[i][1] > heap[k][1]:
            heap[i], heap[k] = heap[k], heap[i]
            i = k
        # 아니면 힙속성 만족
        else:
            break
    
    
# 마지막 원소부터 올라가면서 힙 속성 회복
# j번 인덱스의 부모는 (j - 1) // 2
def upheap():
    i = len(heap) - 1  # 마지막 원소 인덱스
    while (i - 1) // 2 >= 0:   # i가 0이 아닌 동안..부모를 찾아가며 비교.
        k = (i - 1) // 2  # 부모 노드의 인덱스
        # 추가한 노드의 가중치가 부모 노드의 가중치보다 작다면 swap
        if heap[i][1] < heap[k][1]:
            heap[i], heap[k] = heap[k], heap[i]
            i = k
        else:
            break   

N = int(input())
M = int(input())
adj_lst = [[] for _ in range(N + 1)]
for u, v, c in (map(int, input().split()) for _ in range(M)):
    adj_lst[u].append((v, c))
start, end = map(int, input().split())
D = [1e9] * (N + 1)
prev = [-1] * (N + 1)
D[start] = 0
heap = [(start, 0)]
visit = [0] * (N + 1)
while heap:
    cur, cst = heappop()
    while heap and visit[cur]:
        cur, cst = heappop()
    if visit[cur]:
        break
    visit[cur] = 1
    for u, c in adj_lst[cur]:
        if cst + c < D[u]:
            D[u] = cst + c
            prev[u] = cur
            heappush(u, D[u])
t = end
a = [t]
while prev[t] != -1:
    a.append(prev[t])
    t = prev[t]
print(' -> '.join(map(str, a[::-1])))
print(D[end])

5
8
1 2 2
1 3 3
1 4 1
1 5 10
2 4 2
3 4 1
3 5 1
4 5 3
1 5
1 -> 4 -> 5
4


유향 그래프였다.... 문제 잘 보자.

In [ ]:
# 라이브러리 사용 버전.
import sys, heapq
input = sys.stdin.readline
N = int(input())
M = int(input())
adj_lst = [[] for _ in range(N + 1)]
for u, v, c in (map(int, input().split()) for _ in range(M)):
    adj_lst[u].append((v, c))
start, end = map(int, input().split())
D = [1e9] * (N + 1)
D[start] = 0
heap = [(0, start)]
visit = [0] * (N + 1)
while heap:
    cst, cur = heapq.heappop(heap)
    while heap and visit[cur]:
        cst, cur = heapq.heappop(heap)
    if visit[cur]:
        break
    visit[cur] = 1
    for u, c in adj_lst[cur]:
        if cst + c < D[u]:
            D[u] = cst + c
            heapq.heappush(heap, (D[u], u))
print(D[end])

## 3. 1753 최단경로
#### 문제
방향그래프가 주어지면 주어진 시작점에서 다른 모든 정점으로의 최단 경로를 구하는 프로그램을 작성하시오.  
단, 모든 간선의 가중치는 10 이하의 자연수이다.  
  
#### 입력
첫째 줄에 정점의 개수 V와 간선의 개수 E가 주어진다.(1≤V≤20,000, 1≤E≤300,000)모든 정점에는 1부터 V까지 번호가 매겨져 있다고 가정한다.   
둘째 줄에는 시작 정점의 번호 K(1≤K≤V)가 주어진다.  
셋째 줄부터 E개의 줄에 걸쳐 각 간선을 나타내는 세 개의 정수 (u, v, w)가 순서대로 주어진다.  
이는 u에서 v로 가는 가중치 w인 간선이 존재한다는 뜻이다. u와 v는 서로 다르며 w는 10 이하의 자연수이다.  
서로 다른 두 정점 사이에 여러 개의 간선이 존재할 수도 있음에 유의한다.  
  
#### 출력
첫째 줄부터 V개의 줄에 걸쳐, i번째 줄에 i번 정점으로의 최단 경로의 경로값을 출력한다.  
시작점 자신은 0으로 출력하고, 경로가 존재하지 않는 경우에는 INF를 출력하면 된다.  
  
#### 예제 입력 1 
```
5 6
1
5 1 1
1 2 2
1 3 3
2 3 4
2 4 5
3 4 6
```
#### 예제 출력 1 
```
0
2
3
7
INF
```

#### ------------------------------------------------------------------------------------------------------------------------------------------------------------------------

얘도 그냥 다익스트라

In [23]:
import heapq
V, E = map(int, input().split())
K = int(input())
adj_lst = [[] for _ in range(V + 1)]
for u, v, w in (map(int, input().split()) for _ in range(E)):
    adj_lst[u].append((v, w))
D = [1e9] * (V + 1)
D[K] = 0
heap = [(0, K)]
visit = [0] * (V + 1)
while heap:
    cst, cur = heapq.heappop(heap)
    while heap and visit[cur]:
        cst, cur = heapq.heappop(heap)
    if visit[cur]:
        break
    visit[cur] = 1
    for u, c in adj_lst[cur]:
        if cst + c < D[u]:
            D[u] = cst + c
            heapq.heappush(heap, (D[u], u))
print('\n'.join(map(str, (D[i] if D[i] != 1e9 else 'INF' for i in range(1, V + 1) ))))
# for i in range(1, V + 1):
#     print(D[i] if D[i] != 1e9 else 'INF')

5 6
1
5 1 1
1 2 2
1 3 3
2 3 4
2 4 5
3 4 6
0
2
3
7
INF


## 4. 1922 네트워크 연결
#### 문제
도현이는 컴퓨터와 컴퓨터를 모두 연결하는 네트워크를 구축하려 한다.  
하지만 아쉽게도 허브가 있지 않아 컴퓨터와 컴퓨터를 직접 연결하여야 한다.  
그런데 모두가 자료를 공유하기 위해서는 모든 컴퓨터가 연결이 되어 있어야 한다.  
(a와 b가 연결이 되어 있다는 말은 a에서 b로의 경로가 존재한다는 것을 의미한다.  
a에서 b를 연결하는 선이 있고, b와 c를 연결하는 선이 있으면 a와 c는 연결이 되어 있다.)  
  
그런데 이왕이면 컴퓨터를 연결하는 비용을 최소로 하여야 컴퓨터를 연결하는 비용 외에 다른 곳에 돈을 더 쓸 수 있을 것이다.  
이제 각 컴퓨터를 연결하는데 필요한 비용이 주어졌을 때 모든 컴퓨터를 연결하는데 필요한 최소비용을 출력하라.  
모든 컴퓨터를 연결할 수 없는 경우는 없다.  
  
#### 입력
첫째 줄에 컴퓨터의 수 N (1 ≤ N ≤ 1000)가 주어진다.  
  
둘째 줄에는 연결할 수 있는 선의 수 M (1 ≤ M ≤ 100,000)가 주어진다.  
  
셋째 줄부터 M+2번째 줄까지 총 M개의 줄에 각 컴퓨터를 연결하는데 드는 비용이 주어진다.  
이 비용의 정보는 세 개의 정수로 주어지는데,  
만약에 a b c 가 주어져 있다고 하면 a컴퓨터와 b컴퓨터를 연결하는데 비용이 c (1 ≤ c ≤ 10,000) 만큼 든다는 것을 의미한다.  
  
#### 출력
모든 컴퓨터를 연결하는데 필요한 최소비용을 첫째 줄에 출력한다.  
  
#### 예제 입력 1 
```
6
9
1 2 5
1 3 4
2 3 2
2 4 7
3 4 6
3 5 11
4 5 3
4 6 8
5 6 8
```
#### 예제 출력 1 
```
23
```

#### ------------------------------------------------------------------------------------------------------------------------------------------------------------------------